In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random

import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings(action='ignore')

from pycaret.classification import *

import random as python_random
np.random.seed(42)
python_random.seed(42)

In [2]:
PATH = './data/'

train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_qua = pd.read_csv(PATH+'train_quality_data.csv')
train_prob = pd.read_csv(PATH+'train_problem_data.csv')

test_err  = pd.read_csv(PATH+'test_err_data.csv')
test_qua  = pd.read_csv(PATH+'test_quality_data.csv')

sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [6]:
train_err

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0
...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,1
16554659,24999,20201130172625,model_3,05.15.2138,16,1
16554660,24999,20201130172625,model_3,05.15.2138,4,0
16554661,24999,20201130172631,model_3,05.15.2138,4,0


In [46]:
train_err.groupby(by=['fwver','model_nm']).count().sort_values('fwver')

,,user_id,time,errtype,errcode
fwver,model_nm,,,,
03.11.1141,model_4,29,29,29,29
03.11.1149,model_4,2114,2114,2114,2114
03.11.1167,model_4,1505659,1505659,1505659,1505659
04.16.2641,model_1,296,296,296,296
04.16.3345,model_1,101,101,101,101
04.16.3439,model_1,142,142,142,142
04.16.3553,model_1,5237816,5237816,5237816,5237816
04.16.3569,model_1,980,980,980,980
04.16.3571,model_1,145156,145156,145156,145156


In [51]:
train_qua.groupby(by=['user_id', 'fwver']).count().sort_index()

,,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
user_id,fwver,,,,,,,,,,,,,,
10000,05.15.2138,4,4,4,4,4,4,4,4,4,4,4,4,4,4
10002,05.15.2138,22,22,22,22,22,22,22,22,22,22,22,22,22,22
10004,04.22.1750,6,6,6,6,6,6,6,6,6,6,6,6,6,6
10005,04.22.1750,6,6,6,6,6,6,6,6,6,6,6,6,6,6
10006,04.16.3553,2,2,2,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24992,04.22.1750,9,9,9,9,9,9,9,9,9,9,9,9,9,9
24993,05.15.2138,6,6,6,6,6,6,6,6,6,6,6,6,6,6
24995,04.33.1261,6,6,6,6,6,6,6,6,6,6,6,6,6,6


In [57]:
model_ver = train_err[['model_nm','fwver']].drop_duplicates()

In [50]:
train_qua = train_qua.drop_duplicates()

In [62]:
train_qua = train_qua.merge(model_ver, on='fwver', how='left')
train_qua['model_ver'] = train_qua['model_nm']+train_qua['fwver']
train_qua.columns = ['time', 'user_id', 'model_ver', 'quality_0', 'quality_1', 'quality_2',
       'quality_3', 'quality_4', 'quality_5', 'quality_6', 'quality_7',
       'quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,model_nm_x,model_nm_y,model_nm
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,model_3,model_3,model_3
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,4,0,0,0,0,4,0,0,model_3,model_3,model_3
2,20201130210000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,8,0,0,model_3,model_3,model_3
3,20201130210000,10000,05.15.2138,0.0,0,0.0,0,0,8,0,0,0,0,8,0,0,model_3,model_3,model_3
4,20201104110000,10002,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,1,0,0,0,model_3,model_3,model_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284197,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0,model_0,model_0,model_0
284198,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,2,0,0,0,0,17,0,0,model_0,model_0,model_0
284199,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,1,0,0,0,0,17,0,0,model_0,model_0,model_0
284200,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,3,0,0,0,0,17,0,0,model_0,model_0,model_0


In [21]:
train_prob.groupby('user_id').count().sort_values('time', ascending=False)

,time
user_id,
24407,5
15134,4
11016,4
20636,4
24138,4
...,...
15182,1
15179,1
15178,1


In [22]:
train_err[train_err.user_id == 24407]

,user_id,time,model_nm,fwver,errtype,errcode
15683785,24407,20201101063617,model_1,04.16.3553,26,1
15683786,24407,20201101063617,model_1,04.16.3553,16,1
15683787,24407,20201101063618,model_1,04.16.3553,31,1
15683788,24407,20201101063627,model_1,04.16.3553,40,1
15683789,24407,20201101063630,model_1,04.16.3553,40,0
...,...,...,...,...,...,...
15685690,24407,20201130225753,model_2,04.33.1261,40,1
15685691,24407,20201130225754,model_2,04.33.1261,40,0
15685692,24407,20201130225801,model_2,04.33.1261,31,0
15685693,24407,20201130231619,model_2,04.33.1261,23,active


In [33]:
train_err[train_err.errcode == 'active']['errtype'].unique()

array([23], dtype=int64)

In [40]:
train_err.groupby('errcode').count().sort_values('time', ascending=False)[:30]

,user_id,time,model_nm,fwver,errtype
errcode,,,,,
1,8906967,8906967,8906967,8906967,8906967
0,2599123,2599123,2599123,2599123,2599123
connection timeout,1835600,1835600,1835600,1835600,1835600
B-A8002,897863,897863,897863,897863,897863
80,334018,334018,334018,334018,334018
79,332422,332422,332422,332422,332422
14,263577,263577,263577,263577,263577
active,219238,219238,219238,219238,219238
2,166382,166382,166382,166382,166382


In [26]:
train_err.groupby(by=['user_id']).count().sort_values('time', ascending=False)[:20]

,time,model_nm,fwver,errtype,errcode
user_id,,,,,
24934,222186,222186,222186,222186,222186
20546,195538,195538,195538,195538,195538
15570,175730,175730,175730,175730,175730
20352,83680,83680,83680,83680,83680
11041,66093,66093,66093,66093,66093
10648,52284,52284,52284,52284,52284
14724,51588,51588,51588,51588,51588
11367,49933,49933,49933,49933,49933
16439,48784,48784,48784,48784,48784


In [27]:
train_err.groupby(by=['errtype']).count().sort_values('time', ascending=False)[:20]

,user_id,time,model_nm,fwver,errcode
errtype,,,,,
23,2276515,2276515,2276515,2276515,2276515
31,1985294,1985294,1985294,1985294,1985294
22,1865361,1865361,1865361,1865361,1865361
15,1604951,1604951,1604951,1604951,1604951
4,1296750,1296750,1296750,1296750,1296750
16,1249376,1249376,1249376,1249376,1249376
32,1130564,1130564,1130564,1130564,1130564
5,952920,952920,952920,952920,952919
40,730378,730378,730378,730378,730378


In [ ]:
id_error = train_err[['user_id','errtype']].values

train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

In [ ]:
problem = np.zeros(15000)

problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

In [ ]:
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

id_error = test_err[['user_id','errtype']].values
test_error = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    test_error[person_idx - test_user_id_min,err - 1] += 1
test_error = test_error.reshape(test_error.shape[0],-1)
print(test_error.shape)

In [ ]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(error, problem, test_size=0.25, random_state=42)

train_X.shape, test_X.shape, train_y.shape, test_y.shape

In [ ]:
%%time
clf = setup(session_id=seed, 
            data = train, target = 'voted'
          , categorical_features=col_cat
          , numeric_features = ['wr','wf']#,,'tp17','tp4','Q_A15','Q_A3','Q_A15tp17','Q_A3tp4']#['Q_A1','Q_A3','Q_A5','tp17','tp4','Q_A15tp17','Q_A3tp4']
          , ignore_features =['QhE','QlE','QrE'
                              ,'QdA', 'QeA', 'QgA', 'gender', 'QnA','QgE'] +tp.tolist() + wr.tolist() + wf.tolist()#'religion', 'race_1', 'race_2', 'race_3','race_4', 'race_5', 'race_6']
           )

In [ ]:
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True

In [ ]:
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }

In [ ]:
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 3
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

In [ ]:
print(np.mean(auc_scores))

In [ ]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
display(test_err.head())

In [ ]:
# 데이터 설명을 확인하면
# test 데이터는 ueser_id가 30000부터 44998까지 총 14999개가 존재.
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [ ]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

In [ ]:
# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict(test_x)
    print(pred_y)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [ ]:
pred_y_list

In [ ]:
pred_ensemble

In [ ]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [ ]:
sample_submssion['problem'] = pred_ensemble.reshape(-1)

In [ ]:
sample_submssion.to_csv("./submission/submission_20210114-0.csv", index = False)
sample_submssion